<a href="https://colab.research.google.com/github/fv050795/modelagem-de-dados-/blob/main/UNIDADE1_SESS%C3%83O_3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# Criar conexão em memória (simulação de banco)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# ------------------ OLTP ------------------
# Criar tabelas (simulação de sistema transacional)
cursor.execute("""
CREATE TABLE Clientes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT,
    cidade TEXT
)
""")

cursor.execute("""
CREATE TABLE Vendas (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    cliente_id INTEGER,
    produto TEXT,
    quantidade INTEGER,
    preco REAL,
    data DATE,
    FOREIGN KEY(cliente_id) REFERENCES Clientes(id)
)
""")

# Inserir clientes (transações OLTP)
clientes = [
    ("João", "São Paulo"),
    ("Maria", "Rio de Janeiro"),
    ("Pedro", "Belo Horizonte"),
    ("Ana", "São Paulo")
]
cursor.executemany("INSERT INTO Clientes (nome, cidade) VALUES (?,?)", clientes)

# Inserir vendas (transações OLTP)
vendas = [
    (1, "Notebook", 1, 3500.00, "2025-08-01"),
    (2, "Smartphone", 2, 2000.00, "2025-08-02"),
    (3, "Headset", 3, 150.00, "2025-08-02"),
    (1, "Mouse", 2, 80.00, "2025-08-03"),
    (4, "Notebook", 1, 3600.00, "2025-08-03"),
    (2, "Monitor", 1, 800.00, "2025-08-04")
]
cursor.executemany("INSERT INTO Vendas (cliente_id, produto, quantidade, preco, data) VALUES (?,?,?,?,?)", vendas)

conn.commit()

# ------------------ OLAP ------------------
# Total de vendas por cidade
query1 = """
SELECT c.cidade, SUM(v.quantidade * v.preco) as total_vendas
FROM Vendas v
JOIN Clientes c ON v.cliente_id = c.id
GROUP BY c.cidade
"""
df1 = pd.read_sql_query(query1, conn)

# Produto mais vendido em quantidade
query2 = """
SELECT produto, SUM(quantidade) as total_qtd
FROM Vendas
GROUP BY produto
ORDER BY total_qtd DESC
LIMIT 1
"""
df2 = pd.read_sql_query(query2, conn)

# Vendas por dia
query3 = """
SELECT data, SUM(quantidade * preco) as faturamento_dia
FROM Vendas
GROUP BY data
ORDER BY data
"""
df3 = pd.read_sql_query(query3, conn)

# ------------------ Resultados ------------------
print("📊 Total de Vendas por Cidade (OLAP):")
display(df1)

print("\n🏆 Produto mais vendido:")
display(df2)

print("\n📅 Faturamento por Dia:")
display(df3)

# Gráfico de faturamento por dia
plt.figure(figsize=(6,4))
plt.bar(df3['data'], df3['faturamento_dia'])
plt.title("Faturamento por Dia (OLAP)")
plt.xlabel("Data")
plt.ylabel("Faturamento (R$)")
plt.xticks(rotation=45)
plt.show()

# Fechar conexão
conn.close()
